## Cargando la data HAR

https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones

Función para leer un archivo como numpy array

In [ ]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values


función que recorre una lista de archivos y retorna un numpy array 3d. La función `dstack()` de NumPy nos permite apilar cada una de las matrices 3D cargadas en una única matriz 3D donde las variables se separan en la tercera dimensión (características).

In [ ]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded

Podemos usar esta función para cargar todos los datos de la señal de entrada para un grupo dado, como train o test.
La función `load_dataset_group()` a continuación carga todos los datos de la señal de entrada y los datos de salida para un solo grupo(train o test) usando las convenciones de nomenclatura consistentes entre los directorios de test y train.

In [ ]:
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

Finalmente, podemos cargar cada uno de los conjuntos de datos de test y train.

La función `load_dataset()` a continuación implementa este comportamiento y retorna la X e y (para train y test) listos para ajustar y evaluar los modelos definidos.


In [ ]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    #https://www.kaggle.com/srikanthreddyt/human-activity-recognition-using-conv1d
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + './UCI_HAR_Dataset/')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', prefix + './UCI_HAR_Dataset/')
    print(testX.shape, testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return (trainX, trainy), (testX, testy)

Ahora se tiene la data lista para ser usada en un modelo CNN 1D.